In [6]:
import numpy as np
import control as ctrl
import matplotlib.pyplot as plt
from scipy import signal

# System parameters
M = 0.5
m = 0.2
b = 0.1
I = 0.006
g = 9.81
l = 0.3

# Compute system matrix elements
A_22 = (-(I+m*l**2)*b)/(I*(M+m)+M*m*l**2)
A_23 = (m**2 * g * l**2)/ (I*(M+m) + M*m* l**2)
A_42 = (-m*l*b)/(I*(M+m) + M*m* l**2)
A_43 = ((m*g*l)*(M+m))/(I*(M+m) + M*m* l**2)
B_2 = (I + m*l**2)/(I*(M+m) + M*m*l**2)
B_4 = (m*l)/(I*(M+m) + M*m*l**2)

# State space matrices
A = np.array([[0, 1, 0, 0],
              [0, A_22, A_23, 0],
              [0, 0, 0, 1],
              [0, A_42, A_43, 0]])

B = np.array([[0],
              [B_2],
              [0],
              [B_4]])

C = np.array([[1, 0, 0, 0],
              [0, 0, 1, 0]])

D = np.array([[0],
              [0]])

# Create continuous state space system
sys_ss = ctrl.StateSpace(A, B, C, D)

# Discretize the system
Ts = 1/100
sys_d = ctrl.sample_system(sys_ss, Ts, method='zoh')

# Test controllability and observability
Ad = sys_d.A
Bd = sys_d.B
Cd = sys_d.C
Dd = sys_d.D

co = ctrl.ctrb(Ad, Bd)
ob = ctrl.obsv(Ad, Cd)

controllability = np.linalg.matrix_rank(co)
observability = np.linalg.matrix_rank(ob)

print(f"Controllability: {controllability}")
print(f"Observability: {observability}")

# LQR Control
Q = Cd.T @ Cd
R = np.array([[1]])

# Discrete LQR
K, _, _ = ctrl.dlqr(Ad, Bd, Q, R)

# Closed-loop system
Ac = Ad - Bd @ K
Bc = Bd
Cc = Cd
Dc = Dd

sys_cl = ctrl.StateSpace(Ac, Bc, Cc, Dc, Ts)

# Simulation
t = np.arange(0, 5.01, 0.01)
r = 0.2 * np.ones((1, len(t)))

_, y, x = ctrl.forced_response(sys_cl, t, r.T)

# Plot results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

ax1.plot(t, y[0, :])
ax1.set_ylabel('Cart position (m)')

ax2.plot(t, y[1, :])
ax2.set_ylabel('Pendulum angle (radians)')
ax2.set_xlabel('Time (s)')

plt.suptitle('Step Response with Digital LQR Control')
plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'control'